In [1]:
import numpy as np
import os
import pandas as pd
import math
import spectrum
from glob import glob
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [2]:
debug = False
use_freq_resp = True
remove_dc = True

### Reading files

In [3]:
dir_name = 'D://Xin Zhang/Sensor/conbination/NewData/All'
file_list = [y for x in os.walk(dir_name) for y in glob(os.path.join(x[0], '*.txt'))]

In [4]:
len(file_list)

601

In [5]:
all_data = []

for j in file_list:
    temp_df = pd.read_csv(j, sep = "\t")
    
    for x in temp_df.columns:
        temp_df[x] = pd.to_numeric(temp_df[x], downcast="float")   

    all_data.append(temp_df)

In [6]:
if(debug):
    ctr = 0 
    for j in range(len(all_data)):
        if('Multi' in file_list[j]):
            if('trough' in file_list[j]):
                peak_index,_ = find_peaks(-all_data[j]['Current (A)'],distance=3500)
                peaks = peak_index/1000
            elif('peak' in file_list[j]):
                peak_index,_ = find_peaks(all_data[j]['Current (A)'],distance=3500)
                peaks = peak_index/1000
                
            plt.plot(all_data[j]['Elapsed Time (s)'], all_data[j]['Current (A)'])
            plt.scatter(peaks, all_data[j]['Current (A)'][peak_index], c='red')
        
            plt.title(file_list[ctr].rsplit('/', 1)[1].rsplit('\\',2)[2])
            plt.xlabel("Time (s)")
            plt.ylabel("Current (A)")
            plt.pause(0.0001)
            ctr += 1
        else:
            plt.plot(all_data[j]['Elapsed Time (s)'], all_data[j]['Current (A)'])       
            plt.title(file_list[ctr].rsplit('/', 1)[1].rsplit('\\',2)[2])
            plt.xlabel("Time (s)")
            plt.ylabel("Current (A)")
            plt.pause(0.0001)
            ctr += 1

### Pre-processing

In [7]:
num_files = len(all_data)
sampling_rate = 1000 # In Hz
length_keep = 3500
num_samples_to_keep_before = 1500
num_samples_to_keep_after = 1500
total_num_samples_to_keep = 3000

num_multi_files = 0
for i in range(num_files):
    if('Multi' in file_list[i]):
        num_multi_files+=1
print('The number of multi-files is ', num_multi_files)
extra_signal_number = (4-1)*2+(5-1)*10+(6-1)*62+(14-1)+(24-1)+(26-1)+(30-1)+(48-1)

new_file_list = [ [] for _ in range(num_files + extra_signal_number)]
reserved_samples = [ [] for _ in range(num_files + extra_signal_number)]
reserved_time = [ [] for _ in range(num_files + extra_signal_number)]
time_to_keep = np.linspace(0,total_num_samples_to_keep, num = total_num_samples_to_keep)


class_all = [ [] for _ in range(num_files + extra_signal_number)]

class1 = ['YES','Yes','yes']
class2 = ['NO','No','no']
class3 = ['ONE','One','one']
class4 = ['Two','TWO','two']

class5 = ['SHAKING','Shaking','shaking']
class6 = ['NODING','Noding','noding']
class7 = ['STRETCH','Stretch','stretch']

class8 = ['Nodding_No']
class9 = ['Nodding_Yes']
class10 = ['Shaking_No']
class11 = ['Shaking_Yes']
# class12 = ['Shaking_One']

x=0
for i in range(num_files):
    class_match = False
    
    temp_data = all_data[i]
    actual_name = file_list[i].rsplit('/', 1)[1].rsplit('.',1)[0].rsplit('\\',2)[1]
    
    # deal with Multi =================================================================================
    if('Multi' in file_list[i]):
        if('trough' in file_list[i]):
            peak_index,_ = find_peaks(-temp_data['Current (A)'],distance=3500)
            if('No_4' in file_list[i]):
                temp_six_peak = temp_data['Current (A)'][peak_index].nsmallest(4).index
            if('No_5' in file_list[i]):
                temp_six_peak = temp_data['Current (A)'][peak_index].nsmallest(5).index
            if('No_6' in file_list[i]):
                temp_six_peak = temp_data['Current (A)'][peak_index].nsmallest(6).index
            if('No_14' in file_list[i]):
                temp_six_peak = temp_data['Current (A)'][peak_index].nsmallest(14).index
            if('No_24' in file_list[i]):
                temp_six_peak = temp_data['Current (A)'][peak_index].nsmallest(24).index
            if('No_26' in file_list[i]):
                temp_six_peak = temp_data['Current (A)'][peak_index].nsmallest(26).index
            if('No_30' in file_list[i]):
                temp_six_peak = temp_data['Current (A)'][peak_index].nsmallest(30).index
            if('No_48' in file_list[i]):
                temp_six_peak = temp_data['Current (A)'][peak_index].nsmallest(48).index
                
        if('peak' in file_list[i]):
            peak_index,_ = find_peaks(temp_data['Current (A)'],distance=3500)
            temp_six_peak = temp_data['Current (A)'][peak_index].nlargest(6).index

        for k in range(len(temp_six_peak)):
            if(temp_six_peak[k]-num_samples_to_keep_before<0):
                print('Too close to the start in', file_list[i].rsplit('/', 1)[1])
                
                fix = num_samples_to_keep_before - temp_six_peak[k]
                samples_to_keep = pd.Series(temp_data['Current (A)'][0])
                for q in range(fix-1):
                    samples_to_keep = samples_to_keep.append(pd.Series(temp_data['Current (A)'][0]), ignore_index=True)
                samples_to_keep = samples_to_keep.append(temp_data['Current (A)'][0:temp_six_peak[k]+num_samples_to_keep_after], ignore_index=True)
                
                print('fixed (start) length is', len(samples_to_keep))
                print('================================================')
                
            elif(temp_six_peak[k]+num_samples_to_keep_after>len(temp_data['Current (A)'])):
                print('Too close to the end in', file_list[i].rsplit('/', 1)[1])
                
                fix = temp_six_peak[k]+num_samples_to_keep_after-len(temp_data['Current (A)'])
                samples_to_keep = temp_data['Current (A)'][temp_six_peak[k]-num_samples_to_keep_before:len(temp_data['Current (A)'])-1]
                for q in range(fix+1):
                    samples_to_keep = samples_to_keep.append(pd.Series(temp_data['Current (A)'][len(temp_data['Current (A)'])-1]), ignore_index=True)
                
                print('fixed (end) length is', len(samples_to_keep))
                print('================================================')
                
            else:
                samples_to_keep = temp_data['Current (A)'][temp_six_peak[k]-num_samples_to_keep_before:temp_six_peak[k]+num_samples_to_keep_after]
            
            reserved_samples[x].append(samples_to_keep)
            reserved_time[x].append(time_to_keep)
            
            new_file_list[x] = file_list[i]
            
            # distinguish classes =============================================================================
            if(any(temp_name in actual_name for temp_name in class1)):
                class_all[x] = 0
                class_match = True
        
            if(any(temp_name in actual_name for temp_name in class2)):
                class_all[x] = 1
                class_match = True

            if(any(temp_name in actual_name for temp_name in class3)):
                class_all[x] = 2
                class_match = True
                
            if(any(temp_name in actual_name for temp_name in class4)):
                class_all[x] = 3
                class_match = True
                
            if(any(temp_name in actual_name for temp_name in class5)):
                class_all[x] = 4
                class_match = True
                
            if(any(temp_name in actual_name for temp_name in class6)):
                class_all[x] = 5
                class_match = True
            
            if(any(temp_name in actual_name for temp_name in class7)):
                class_all[x] = 6
                class_match = True
                
            if(any(temp_name in actual_name for temp_name in class8)):
                class_all[x] = 7
                class_match = True
                
            if(any(temp_name in actual_name for temp_name in class9)):
                class_all[x] = 8
                class_match = True
                
            if(any(temp_name in actual_name for temp_name in class10)):
                class_all[x] = 9
                class_match = True
                
            if(any(temp_name in actual_name for temp_name in class11)):
                class_all[x] = 10
                class_match = True

                
            if(not class_match):
                print("Could not find any class matches for ",actual_name)
                
            x += 1    
            
    # deal with single=================================================================================================
    else:
        new_file_list[x] = file_list[i]
        
        # euqal signal length =============================================================================
        indices_to_keep = temp_data['Current (A)'].idxmin()
        
        if(indices_to_keep-num_samples_to_keep_before<0):
            print('Too close to the start in', file_list[i].rsplit('/', 1)[1])
            
            fix = num_samples_to_keep_before - indices_to_keep
            samples_to_keep = pd.Series(temp_data['Current (A)'][0])
            for q in range(fix-1):
                samples_to_keep = samples_to_keep.append(pd.Series(temp_data['Current (A)'][0]), ignore_index=True)
            samples_to_keep = samples_to_keep.append(temp_data['Current (A)'][0:indices_to_keep+num_samples_to_keep_after], ignore_index=True)
            
            print('fixed (start) length is', len(samples_to_keep))
            print('================================================')
        
        elif(indices_to_keep+num_samples_to_keep_after>len(temp_data['Current (A)'])):
            print('Too close to the end in', file_list[i].rsplit('/', 1)[1])
                
            fix = indices_to_keep+num_samples_to_keep_after-len(temp_data['Current (A)'])
            samples_to_keep = temp_data['Current (A)'][indices_to_keep-num_samples_to_keep_before:len(temp_data['Current (A)'])-1]
            for q in range(fix+1):
                samples_to_keep = samples_to_keep.append(pd.Series(temp_data['Current (A)'][len(temp_data['Current (A)'])-1]), ignore_index=True)
                
            print('fixed (end) length is', len(samples_to_keep))
            print('================================================')
        
        else:
            samples_to_keep = temp_data['Current (A)'][indices_to_keep-num_samples_to_keep_before:indices_to_keep+num_samples_to_keep_after]
            
        reserved_samples[x].append(samples_to_keep)
        reserved_time[x].append(time_to_keep)

            
        # distinguish classes =============================================================================
        if(any(temp_name in actual_name for temp_name in class1)):
            class_all[x] = 0
            class_match = True
        
        if(any(temp_name in actual_name for temp_name in class2)):
            class_all[x] = 1
            class_match = True

        if(any(temp_name in actual_name for temp_name in class3)):
            class_all[x] = 2
            class_match = True
            
        if(any(temp_name in actual_name for temp_name in class4)):
            class_all[x] = 3
            class_match = True
            
        if(any(temp_name in actual_name for temp_name in class5)):
            class_all[x] = 4
            class_match = True
            
        if(any(temp_name in actual_name for temp_name in class6)):
            class_all[x] = 5
            class_match = True
            
        if(any(temp_name in actual_name for temp_name in class7)):
            class_all[x] = 6
            class_match = True
                
        if(any(temp_name in actual_name for temp_name in class8)):
            class_all[x] = 7
            class_match = True
                
        if(any(temp_name in actual_name for temp_name in class9)):
            class_all[x] = 8
            class_match = True
                
        if(any(temp_name in actual_name for temp_name in class10)):
            class_all[x] = 9
            class_match = True
                
        if(any(temp_name in actual_name for temp_name in class11)):
            class_all[x] = 10
            class_match = True
            
        if(not class_match):
            print("Could not find any class matches for ",actual_name)
            
        x += 1
        # =================================================================================================      

The number of multi-files is  79
Too close to the end in All\MOVE-NODING\MOVE-NODING - Copy (31).txt
fixed (end) length is 3000
Too close to the end in All\MOVE-SHAKING\MOVE-SHAKING - Copy (48).txt
fixed (end) length is 3000
Too close to the end in All\Nodding_No_Multi_trough\Nodding_No_6 in 1_5s per file - Copy (10).txt
fixed (end) length is 3000
Too close to the end in All\Nodding_Yes\SPEAK-YES-NODING - 10.txt
fixed (end) length is 3000
Too close to the end in All\Nodding_Yes\SPEAK-YES-NODING - 11.txt
fixed (end) length is 3000
Too close to the end in All\Nodding_Yes\SPEAK-YES-NODING - 146.txt
fixed (end) length is 3000
Too close to the start in All\Shaking_Yes_Multi_trough\Shaking_Yes-No_5 in 1_5s per file - Copy (14).txt
fixed (start) length is 3000
Too close to the start in All\Shaking_Yes_Multi_trough\Shaking_Yes-No_5 in 1_5s per file - Copy (15).txt
fixed (start) length is 3000
Too close to the start in All\Shaking_Yes_Multi_trough\Shaking_Yes-No_6 in 1_5s per file - Copy (17).t

#### Filter

In [8]:
if(debug):
    for i in range(len(reserved_samples)):
        for j in range(len(reserved_samples[i])):
            if(len(reserved_samples[i][j]) == total_num_samples_to_keep):
                plt.plot(reserved_time[i][j], reserved_samples[i][j])
                plt.xlabel("Time (s)")
                plt.ylabel("Current (A)")
                plt.title(new_file_list[i].rsplit('/', 1)[1].rsplit('\\',2)[2])
                plt.pause(0.000001)
            else:
                print('Not enough points in', new_file_list[i].rsplit('/', 1)[1])

In [9]:
from scipy import signal

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

# ==================================================================================

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut/nyq
    high = highcut/nyq
    b, a = signal.butter(order, [low,high], btype='bandpass', analog=False)
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [10]:
overall_array = np.array([])
temp_array = []

for i in range(len(reserved_samples)):
    for j in range(len(reserved_samples[i])):
        if(remove_dc):
            filtered_wave = butter_bandpass_filter(reserved_samples[i][j], 1, 300, sampling_rate, 2)
            # filtered_wave = butter_highpass_filter(reserved_samples[i][j], 0.01, sampling_rate, 2)
            if(debug):
                plt.plot(reserved_time[i][j],filtered_wave)
                plt.title(new_file_list[i].rsplit('/', 1)[1].rsplit('\\',2)[2])
                plt.xlabel("Time (s)")
                plt.ylabel("filtered_wave")
                plt.pause(0.000001)
            if(use_freq_resp):
                temp_data = np.fft.fft(filtered_wave).real
                temp_array = np.expand_dims(np.concatenate([reserved_samples[i][j], temp_data, np.array([class_all[i]])]),0)
                if(debug):
                    plt.plot(reserved_time[i][j], temp_data)
                    plt.ylim((-0.0005, 0.0005))
                    plt.ylabel("FFT")
                    plt.title(new_file_list[i].rsplit('/', 1)[1].rsplit('\\',2)[2])
                    plt.pause(0.000001)
        
            else:
                temp_array = np.expand_dims(np.concatenate([filtered_wave, np.array([class_all[i]])]),0)
                if(debug):
                    plt.plot(reserved_time[i][j], filtered_wave)
                    plt.ylim((-0.00025, 0.00025))
                    plt.ylabel("current")
                    plt.xlabel("time")
                    plt.title(new_file_list[i].rsplit('/', 1)[1].rsplit('\\',2)[2])
                    plt.pause(0.000001)
        else:
            temp_array = np.expand_dims(np.concatenate([reserved_samples[i][j], np.array([class_all[i]])]),0)
            if(debug):
                plt.plot(reserved_time[i][j], reserved_samples[i][j])
                plt.ylabel("current")
                plt.xlabel("time")
                plt.title(new_file_list[i].rsplit('/', 1)[1].rsplit('\\',2)[2])
                plt.pause(0.000001)
                
    if(len(overall_array) == 0):
        overall_array = temp_array      
    else:
        overall_array = np.concatenate([overall_array, temp_array], 0)

In [11]:
overall_array.shape

(1094, 6001)

In [12]:
dir_to_save = 'D://Xin Zhang/Sensor/conbination/processed_data/'
file_name_to_save = "All_data_twoinputs"

# if(not remove_dc):
#     file_name_to_save = file_name_to_save + "_original"
# elif(use_freq_resp):
#     file_name_to_save = file_name_to_save + "_freq_resp"

    
with open(dir_to_save + file_name_to_save + '.npy', 'wb') as f:
    np.save(f, overall_array)
print("Saved to:", dir_to_save +  file_name_to_save + '.npy')

Saved to: D://Xin Zhang/Sensor/conbination/processed_data/All_data_twoinputs.npy
